In [4]:
# imports
import imageio.v2 as imageio
from PIL import Image
from app import S3
import fastdup
import os

bucket: str = "hydrahi4ai"
s3_img_path: str = "s3://hydrahi4ai/ajio-myntra/alternative/20231219/"
local_img_path: str = "trash/fastdup/tmp/" + s3_img_path.split(bucket)[-1]
s3 = S3(bucket)

In [ ]:
fd = fastdup.create("trash/fastdup/")

# analisamos las imagenes del s3 para detectar las images dañadas y tomamos una que este bien para usarla como muestra
fd.run(s3_img_path, threshold= 0.5, overwrite= True, high_accuracy= True)

# Obtenemos los similarity
similarity = fd.similarity()

In [ ]:
# lista de los archivos dañados, para descargarlos
invalid_img_s3: list = fd.invalid_instances()["filename"].to_list()
invalid_img_local: list = [f"{local_img_path}{s3_path.split('/')[-1]}" for s3_path in invalid_img_s3]

In [ ]:
for local_file in invalid_img_local:
    
    try:
        imagen = imageio.imread(local_file)
        
        # Agregar metadata
        metadata = {
            'jfif': 257,
            'jfif_version': (1, 1),
        }

        # re-escribimos la imagen con la metadata agregada
        imageio.imwrite(local_file, imagen, **metadata)
        
    except ValueError as e:
        print(e)

In [ ]:
# No ejecutar  esta celda hasta que estes seguro de que la data esta bien

# subimos los archivos acomodados al S3 

# for local_file in invalid_img_local:
    
#     s3.update(
#         s3_img_path,
#         local_file
#     )
